# Temperature - Basic Usage of HiGlass

In this notebook we will talk through the main aspects of the API and how to get started visualizing data

In [ ]:
HOST = 'localhost'
PORT = 8001

# If using Binder, set this to the current host: https://hub.{domain}/user-{username}-{hash}
LOCATION = ''

if LOCATION:
    # Use the jupyterhub proxy address to reach our server
    API_URL = LOCATION.replace('/lab', '') + f'/proxy/{PORT}/api/v1'
else:
    API_URL = f'http://{HOST}:{PORT}/api/v1'

%load_ext autoreload
%autoreload 2

### Load data

We optained the data from the National Centers for Environmental Information: https://www.ncei.noaa.gov/access/search/data-search/global-hourly

In [ ]:
from utils import get_ncei_temperature_data_as_array

temps_by_city, min_max_dates = get_ncei_temperature_data_as_array({
    "corpus_christi": {
        "files": ["data/1796105.csv.gz"],
        "station": 72251012924
    },
    "boston": {
        "files": [
            "data/1577451.csv.gz",
            "data/1577468.csv.gz",
            "data/1573178.csv.gz"
        ],
        "station": 72509014739
    },
    "fairbanks": {
        "files": ["data/1707246.csv.gz"],
        "station": 70261026411
    }
})

### Create tilesets

In [ ]:
from higlass import Tileset
from clodius.tiles.npvector import tileset_info, tiles_wrapper

tileset_corpus_christi = Tileset(
    tileset_info=lambda: tileset_info(temps_by_city['corpus_christi']),
    tiles=lambda tile_ids: tiles_wrapper(temps_by_city['corpus_christi'], tile_ids),
    name='Corpus Christi',
    
)

tileset_boston = Tileset(
    tileset_info=lambda: tileset_info(temps_by_city['boston']),
    tiles=lambda tile_ids: tiles_wrapper(temps_by_city['boston'], tile_ids),
    name='Boston'
)

tileset_fairbanks = Tileset(
    tileset_info=lambda: tileset_info(temps_by_city['fairbanks']),
    tiles=lambda tile_ids: tiles_wrapper(temps_by_city['fairbanks'], tile_ids),
    name='Fairbanks'
)

### Create HiGlass view

In [ ]:
from higlass import Track, View, display

track_definition = {
    'track_type': 'horizontal-line',
    'height': 100,
    'server': API_URL,
}

corpus_christi_line = Track(tileset=tileset_corpus_christi, **track_definition)
boston_line = Track(tileset=tileset_boston, **track_definition)
fairbanks_line = Track(tileset=tileset_fairbanks, **track_definition)

top_axis = Track('top-axis')

widget, server, viewconf = display(
    [View([top_axis, corpus_christi_line, boston_line, fairbanks_line])],
    host=HOST,
    server_port=PORT
)

widget

### Customize appearance of tracks

In [ ]:
from copy import deepcopy

## Important change ##########
new_track_definition = {
    'options': {
        'lineStrokeWidth': 2,
        'valueScaleMin': -50,
        'valueScaleMax': +50,
        'constIndicators': [
            { 'value': -40, 'label': '-40', 'labelSize': 8, 'opacity': 0.3 },
            { 'value': -20, 'label': '-20', 'labelSize': 8, 'opacity': 0.3 },
            { 'value':   0, 'label':   '0', 'labelSize': 8, 'opacity': 0.3 },
            { 'value':  20, 'label':  '20', 'labelSize': 8, 'opacity': 0.3 },
            { 'value':  40, 'label':  '40', 'labelSize': 8, 'opacity': 0.3 },
        ],
        'axisPositionHorizontal': 'hidden',
        'labelColor': '[glyph-color]',
        'labelTextOpacity': 1.0,
        'labelPosition': 'topRight',
    }
}
new_track_definition.update(track_definition)
##############################

track_def_cc = deepcopy(new_track_definition)
track_def_cc['options']['lineStrokeColor'] = 'orange'
corpus_christi_line = Track(tileset=tileset_corpus_christi, **track_def_cc)

track_def_bos = deepcopy(new_track_definition)
track_def_bos['options']['lineStrokeColor'] = 'mediumseagreen'
boston_line = Track(tileset=tileset_boston, **track_def_bos)

track_def_fb = deepcopy(new_track_definition)
track_def_fb['options']['lineStrokeColor'] = 'blue'
fairbanks_line = Track(tileset=tileset_fairbanks, **track_def_fb)

widget, _, _ = display(
    [View([top_axis, corpus_christi_line, boston_line, fairbanks_line])],
    host=HOST,
    server_port=PORT
)

widget

### Combine tracks

In [ ]:
from higlass import CombinedTrack

corpus_christi_line = corpus_christi_line.change_options(labelRightMargin=116)
boston_line = boston_line.change_options(labelRightMargin=64)

## Important change ##########
combined_track = CombinedTrack(
    position='top',
    height=200,
    tracks=[corpus_christi_line, boston_line, fairbanks_line]
)
##############################

widget, _, _ = display(
    [View([top_axis, combined_track])],
    host=HOST,
    server_port=PORT
)

widget

### Different rendering paradigms for the same data

In [ ]:
## Important change ##########
heatmap_settings = {
    'track_type': 'horizontal-1d-heatmap',
    'position': 'top',
    'height': 48,
    'options': {
        'colorRange': ['#003aa8', '#3379ff', '#dddddd', '#ff3d33', '#990700'],
        'valueScaleMin': -20,
        'valueScaleMax': +20,
    }
}

corpus_christi_heatmap = Track(tileset=tileset_corpus_christi, **heatmap_settings)
boston_heatmap = Track(tileset=tileset_boston, **heatmap_settings)
fairbanks_heatmap = Track(tileset=tileset_fairbanks, **heatmap_settings)
##############################

view = {
    'x': 0,
    'y': 1,
    ## Important change ##########
    'initialXDomain': [539148, 565428], # Winters 2014/15, 2015/16, 2016/17
    ##############################
    'tracks': [
        top_axis,
        corpus_christi_heatmap, boston_heatmap, fairbanks_heatmap,
        combined_track
    ]
}


widget, _, _ = display([View(**view)], host=HOST, server_port=PORT)

widget

Winters 2014/15, 2015/16, and 2016/17

---

**Some basic stats about the temperature data**

In [ ]:
import numpy as np

print('Temperature measurements from {} to {}'.format(*min_max_dates))

print('Temperatures in Corpus Christi range from {}°C to {}°C'.format(
    np.nanmin(temps_by_city['corpus_christi']), np.nanmax(temps_by_city['corpus_christi'])
))

print('Temperatures in Boston range from {}°C to {}°C'.format(
    np.nanmin(temps_by_city['boston']), np.nanmax(temps_by_city['boston'])
))

print('Temperatures in Fairbanks range from {}°C to {}°C'.format(
    np.nanmin(temps_by_city['fairbanks']), np.nanmax(temps_by_city['fairbanks'])
))